In [1]:
import pandas as pd
import numpy as np

In [2]:
data = pd.read_csv('../data/train.csv')

In [3]:
data.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000


In [4]:
data.shape

(1460, 81)

# Remove columns with high amount of null values

In [5]:
null_cols = data.isna().sum()
null_cols.loc[null_cols>0]

LotFrontage      259
Alley           1369
MasVnrType         8
MasVnrArea         8
BsmtQual          37
BsmtCond          37
BsmtExposure      38
BsmtFinType1      37
BsmtFinType2      38
Electrical         1
FireplaceQu      690
GarageType        81
GarageYrBlt       81
GarageFinish      81
GarageQual        81
GarageCond        81
PoolQC          1453
Fence           1179
MiscFeature     1406
dtype: int64

In [6]:
null_cols_per = null_cols[null_cols.gt(0)]/len(data)*100
null_cols_per

LotFrontage     17.739726
Alley           93.767123
MasVnrType       0.547945
MasVnrArea       0.547945
BsmtQual         2.534247
BsmtCond         2.534247
BsmtExposure     2.602740
BsmtFinType1     2.534247
BsmtFinType2     2.602740
Electrical       0.068493
FireplaceQu     47.260274
GarageType       5.547945
GarageYrBlt      5.547945
GarageFinish     5.547945
GarageQual       5.547945
GarageCond       5.547945
PoolQC          99.520548
Fence           80.753425
MiscFeature     96.301370
dtype: float64

In [7]:
drop_cols = list(null_cols_per[null_cols_per > 45].index)
data = data.drop(drop_cols, axis=1)

# Look into and drop columns with low variance

In [8]:
low_variance = []
 
for col in data._get_numeric_data():
    minimum = min(data[col])
    ninety_perc = np.percentile(data[col], 90)
    if ninety_perc == minimum:
        low_variance.append(col)
 
print(low_variance)

['LowQualFinSF', 'BsmtHalfBath', '3SsnPorch', 'ScreenPorch', 'PoolArea', 'MiscVal']


In [14]:
data['CombinedIntPorch'] = data["EnclosedPorch"] + data["3SsnPorch"] + data["ScreenPorch"]
data['CombinedIntPorch']

0         0
1         0
2         0
3       272
4         0
       ... 
1455      0
1456      0
1457      0
1458    112
1459      0
Name: CombinedIntPorch, Length: 1460, dtype: int64

In [15]:
data = data.drop(low_variance, axis=1)

# Look into and drop columns with very few unique values

In [16]:
unique_vals = data.nunique()
unique_vals

Id                  1460
MSSubClass            15
MSZoning               5
LotFrontage          110
LotArea             1073
                    ... 
YrSold                 5
SaleType               9
SaleCondition          6
SalePrice            663
CombinedIntPorch     164
Length: 71, dtype: int64

In [17]:
unique_vals.loc[unique_vals <= 2]

Street        2
Utilities     2
CentralAir    2
dtype: int64

In [18]:
data.Street.value_counts()

Pave    1454
Grvl       6
Name: Street, dtype: int64

In [19]:
data.Utilities.value_counts()

AllPub    1459
NoSeWa       1
Name: Utilities, dtype: int64

In [20]:
data.CentralAir.value_counts()

Y    1365
N      95
Name: CentralAir, dtype: int64

In [21]:
data.drop(columns=["Street", "Utilities"])

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,LotShape,LandContour,LotConfig,LandSlope,Neighborhood,...,PavedDrive,WoodDeckSF,OpenPorchSF,EnclosedPorch,MoSold,YrSold,SaleType,SaleCondition,SalePrice,CombinedIntPorch
0,1,60,RL,65.0,8450,Reg,Lvl,Inside,Gtl,CollgCr,...,Y,0,61,0,2,2008,WD,Normal,208500,0
1,2,20,RL,80.0,9600,Reg,Lvl,FR2,Gtl,Veenker,...,Y,298,0,0,5,2007,WD,Normal,181500,0
2,3,60,RL,68.0,11250,IR1,Lvl,Inside,Gtl,CollgCr,...,Y,0,42,0,9,2008,WD,Normal,223500,0
3,4,70,RL,60.0,9550,IR1,Lvl,Corner,Gtl,Crawfor,...,Y,0,35,272,2,2006,WD,Abnorml,140000,272
4,5,60,RL,84.0,14260,IR1,Lvl,FR2,Gtl,NoRidge,...,Y,192,84,0,12,2008,WD,Normal,250000,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1455,1456,60,RL,62.0,7917,Reg,Lvl,Inside,Gtl,Gilbert,...,Y,0,40,0,8,2007,WD,Normal,175000,0
1456,1457,20,RL,85.0,13175,Reg,Lvl,Inside,Gtl,NWAmes,...,Y,349,0,0,2,2010,WD,Normal,210000,0
1457,1458,70,RL,66.0,9042,Reg,Lvl,Inside,Gtl,Crawfor,...,Y,0,60,0,5,2010,WD,Normal,266500,0
1458,1459,20,RL,68.0,9717,Reg,Lvl,Inside,Gtl,NAmes,...,Y,366,0,112,4,2010,WD,Normal,142125,112


# Check data types

In [24]:
data.dtypes

Id                    int64
MSSubClass            int64
MSZoning             object
LotFrontage         float64
LotArea               int64
                     ...   
YrSold                int64
SaleType             object
SaleCondition        object
SalePrice             int64
CombinedIntPorch      int64
Length: 71, dtype: object

In [25]:
data = data.to_csv('../data/train_clean.csv', index=False)